In [1]:
# import xml.etree.ElementTree as ET
import requests
import os
import pandas as pd


In [3]:
id = "010594"

In [2]:
df = pd.read_csv("data/법령검색목록_법령_건축.csv", skiprows=1)
df.head()

,순번,법령MST,소관부처코드,소관부처명,법령ID,법령명,공포일자,공포번호,시행일자,법령구분코드,법령구분명,법령분야코드,법령분야명
0,1,276923,1613000,국토교통부,10594,건축기본법,20251001,제21065호,20251001,A0002,법률,34030000,第3章 建 築
1,2,277915,1613000,국토교통부,10787,건축기본법 시행령,20251001,제35811호,20251001,A0007,대통령령,34030000,第3章 建 築
2,3,266691,1613000,국토교통부,13478,건축물관리법,20241203,제20549호,20250604,A0002,법률,34030000,第3章 建 築
3,4,271337,1613000,국토교통부,13742,건축물관리법 시행령,20250527,제35549호,20250604,A0007,대통령령,34030000,第3章 建 築
4,5,271531,1613000,국토교통부,13747,건축물관리법 시행규칙,20250602,제1495호,20250602,A0103,국토교통부령,34030000,第3章 建 築


In [3]:
df.keys() # 법령

Index(['순번', '법령MST', '소관부처코드', '소관부처명', '법령ID', '법령명', '공포일자', '공포번호', '시행일자',
       '법령구분코드', '법령구분명', '법령분야코드', '법령분야명'],
      dtype='object')

In [4]:
id = df["법령ID"][0]
id

np.int64(10594)

In [120]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙법률 제21065호(2025.10.1)', '정부조직법 일부개정법률', '[본문 생략]', '        부칙', '제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터ㆍㆍㆍ<생략>ㆍㆍㆍ 시행한다.', '  1. 및 2. 생략', '제2조부터 제6조까지 생략', '제7조(다른 법률의 개정) ①부터 <436>까지 생략', '  <437> 건축기본법 일부를 다음과 같이 개정한다.', '  제20조 각 호 외의 부분 및 제21조제1항ㆍ제2항 중 "산업통상자원부장관"을 각각 "산업통상부장관"으로 한다.', '  <438>부터 <626>까지 생략', '제8조 생략']]}, '법령키': '0105942025100121065', '기본정보': {'법령명_한글': '건축기본법', '공포번호': '21065', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010594', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '법률', '법종구분코드': 'A0002'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '388805', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': [{'부칙키': '2007122108783', '부칙공포일자': '20071221', '부칙내용': [['부칙 <제8783호,20

In [122]:
data["법령"].keys()

dict_keys(['개정문', '법령키', '기본정보', '부칙', '조문', '제개정이유'])

In [126]:
data["법령"]["조문"]["조문단위"] # 본문

[{'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001000',
  '조문내용': '            제1장 총칙',
  '조문제목': '',
  '조문이동이후': '',
  '조문여부': '전문'},
 {'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001001',
  '조문내용': '제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
  '조문제목': '목적',
  '조문이동이후': '',
  '조문여부': '조문'},
 {'조문번호': '2',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0002001',
  '항': {'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
    {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
    {'호번호': '3.',
     '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]},
  '조문내용': '제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.',
  '조문제목': '기본이념',
  '조문이동이후': '',
  '조문여부': '조문'},
 {'조문번호': '3',
  '조문제개정

# 자치법규

- 요청변수

    |요청변수|값|설명|
    |---|---|---|
    |OC	|string(필수)|사용자 이메일의 ID|
    |type|char(필수)|출력 형태 HTML/XML/JSON|
    |target|string : ordin(필수)|서비스 대상|
    |ID|char|자치법규ID|
    |MST|string|자치법규 일련번호|
\
- 특이사항
    - '조문여부': 'N' 이지만 '조문내용'은 있는 경우는 보통 조항 제목이다. 예시: '제6장 공개공지 등'

In [4]:
df = pd.read_csv("data/법령검색목록_자치법규_건축.csv", skiprows=1)
df.head()

,순번,자치법규MST,자치법규명,지자체명,자치법규ID,제개정구분,법령종류,공포번호,공포일자,시행일
0,1,1298118,가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례,경기도 가평군,2019610,일부개정,조례,제2623호,20170731,20170731
1,2,1888393,가평군 건축물관리 조례,경기도 가평군,2205553,일부개정,조례,제3174호,20231226,20231226
2,3,2028113,가평군 건축 조례,경기도 가평군,2019611,일부개정,조례,제3296호,20250409,20250409
3,4,1967195,가평군 건축 조례 시행규칙,경기도 가평군,2019612,일부개정,규칙,제1499호,20240919,20240919
4,5,2015645,가평군 군계획 조례,경기도 가평군,2019668,일부개정,조례,제3281호,20250217,20250217


In [5]:
df.keys() # 자치법규

Index(['순번', '자치법규MST', '자치법규명', '지자체명', '자치법규ID', '제개정구분', '법령종류', '공포번호',
       '공포일자', '시행일'],
      dtype='object')

In [6]:
id = df["자치법규ID"][1602]
id

np.int64(2233372)

In [7]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'LawService': {'자치법규기본정보': {'자치법규ID': '2233372', '자치법규명': '횡성군 수도급수 조례', '자치법규일련번호': '2058483', '지자체기관명': '강원특별자치도 횡성군', '시행일자': '20250714', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2855', '전화번호': '033-340-5821', '담당부서명': '상하수도사업소', '자치법규발의종류': '', '공포일자': '20250714'}, '별표': {'별표단위': {'별표제목': '별표', '별표첨부파일구분': 'hwp', '별표첨부파일명': 'http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=154521627&flNm=%EB%B3%84%ED%91%9C', '별표번호': '0001', '별표키': '20574529', '별표내용': '', '별표구분': '서식', '별표가지번호': '00'}}, '부칙': {'부칙공포일자': '20250714', '부칙내용': '부칙  ①(시행일) 이 조례는 공포한 날로부터 시행한다.②(폐지조례) 이 조례 시행과 동시 횡성군수도급수조례(1979.6.23 조례제647호)는 이를 폐지한다.③(경과조치) 이 조례 시행전에 종전 조례에 의하여 행하여진 승인 및 처분사항은 이 조례에 의한 것으로 본다.④(가산금 등에 대한 경과조치) 이 조례 시행전에 검침 및 체납된 수도사용요금 및 급수장치 손료가산금에 대하여는 종전의 규정에 의한다부칙 <2001.5.8.> 이 조례는 공포한 날로부터 시행한다.부칙 <2003.1.27.> 제1조(시행일) (생략)제2조(한시기구에 대한 경과규정) (생략)제3조(다른 조례의 개정) 제1항 내지 제11항 (생략)⑫횡성군수도급수조례중 다음과 같이 개정한다.제9조제2항중 “상하수도 담당주사”를 “상수도 담당주사”로 한다.제13항 내지 제18항 (생략)부칙 <2003.6.2.> 이 조례는 공포한 날로부터 시행한다.부칙 <2

In [8]:
print(data.keys())
data["LawService"].keys()

dict_keys(['LawService'])


dict_keys(['자치법규기본정보', '별표', '부칙', '조문'])

In [9]:
meta = data["LawService"]["자치법규기본정보"]
meta

{'자치법규ID': '2233372',
 '자치법규명': '횡성군 수도급수 조례',
 '자치법규일련번호': '2058483',
 '지자체기관명': '강원특별자치도 횡성군',
 '시행일자': '20250714',
 '제개정정보': '일부개정',
 '자치법규종류': 'C0001',
 '공포번호': '2855',
 '전화번호': '033-340-5821',
 '담당부서명': '상하수도사업소',
 '자치법규발의종류': '',
 '공포일자': '20250714'}

In [10]:
content = data["LawService"]["조문"]["조"]
content

[{'조문번호': ['000000', '000000'], '조제목': '', '조내용': '제1장 총칙', '조문여부': 'N'},
 {'조문번호': ['000100', '000100'],
  '조제목': '목적',
  '조내용': '제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.',
  '조문여부': 'Y'},
 {'조문번호': ['000200', '000200'],
  '조제목': '정의',
  '조내용': '제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음 각 호와 같다.1. "급수설비"란 수도사업자가 일반의 수요자에게 원수 또는 정수를 공급하기 위하여 설치한 배수관으로부터 분기하여 설치된 급수관(옥내급수관을 포함한다)·계량기·저수조·수도꼭지·그 밖의 급수를 위하여 필요한 기구(器具)를 말한다.2. "급수공사"란 급수설비의 신설, 개조, 수선 및 철거 등의 공사를 말한다.3. "수도사용자 등"이란 급수설비의 사용자, 소유자 및 관리인 등을 말한다.4. “중수도”란 사용한 수돗물을 생활용수·공업용수 등으로 재활용할 수 있도록 다시 처리하는 시설을 말한다.5. "공업용수"란 정수장에서 1차 침전 처리한 용수를 말한다.6. “호(戶)”란 독립적으로 생활을 영위할 수 있도록 건립된 건축단위를 말한다.7. “교육시설”이란「초·중등교육법」 제2조 각 호에 따른 학교 및 「유아교육법」 제2조제2호에 따른 유치원에서 사용하는 시설을 말한다.<개정 2019. 7. 15.>',
  '조문여부': 'Y'},
 {'조문번호': ['000300', '000300'],
  '조제목': '급수구역',
  '조내용': '제3조(급수구역)① 급수구역은 횡성군(이하 “군”이라 한다)의 관할구역 중 횡성군수(이하 "군수"라 한다)가 고시한 급수가능구역으로 한다.

In [43]:
ctx_list = []
header = ""
for ctx in content:
    if ctx["조문여부"] == "Y":
        txt = header + " " + ctx["조내용"]
        ctx_list.append(txt.strip())
        # header = ""
    else:
        header = "[" + ctx["조내용"] + "]"
ctx_list

['[제1장 총칙] 제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.',
 '[제1장 총칙] 제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음 각 호와 같다.1. "급수설비"란 수도사업자가 일반의 수요자에게 원수 또는 정수를 공급하기 위하여 설치한 배수관으로부터 분기하여 설치된 급수관(옥내급수관을 포함한다)·계량기·저수조·수도꼭지·그 밖의 급수를 위하여 필요한 기구(器具)를 말한다.2. "급수공사"란 급수설비의 신설, 개조, 수선 및 철거 등의 공사를 말한다.3. "수도사용자 등"이란 급수설비의 사용자, 소유자 및 관리인 등을 말한다.4. “중수도”란 사용한 수돗물을 생활용수·공업용수 등으로 재활용할 수 있도록 다시 처리하는 시설을 말한다.5. "공업용수"란 정수장에서 1차 침전 처리한 용수를 말한다.6. “호(戶)”란 독립적으로 생활을 영위할 수 있도록 건립된 건축단위를 말한다.7. “교육시설”이란「초·중등교육법」 제2조 각 호에 따른 학교 및 「유아교육법」 제2조제2호에 따른 유치원에서 사용하는 시설을 말한다.<개정 2019. 7. 15.>',
 '[제1장 총칙] 제3조(급수구역)① 급수구역은 횡성군(이하 “군”이라 한다)의 관할구역 중 횡성군수(이하 "군수"라 한다)가 고시한 급수가능구역으로 한다. 다만, 군수가 공익상 필요하다고 인정할 때에는 관할구역 이외의 구역에도 급수할 수 있다.② 공업용수의 급수구역은 공업용수 공급시설이 설치된 산업단지에 한한다.③ 급수구역 내에서 식수는 지방상수도를 사용하는 것을 원칙으로 한다.',
 '[제1장 총칙] 제4조(급수설비의 구분) 급수설비는 다음 각 호와 같이 구분한다.1. 전용 급수설비 : 1호 또는 1개소의 급수를 목적으로 설치하는 급수설비2. 공용 급수설비 : 주민의 공동급수를 목적으로

In [12]:
# Qdrant 클라이언트 설치
# pip install qdrant-client

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue

QDRANT_URL = os.getenv('QDRANT_URL', '')
QDRANT_api_key = os.getenv('QDRANT_api_key', '')

# Qdrant 클라이언트 초기화 (로컬 또는 클라우드)
# client = QdrantClient(host="localhost", port=6333)  # 로컬 사용 시
# 또는
# client = QdrantClient(url="https://your-cluster.qdrant.io", api_key="your-api-key")  # 클라우드 사용 시


In [14]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_api_key,
)

print(qdrant_client.get_collections())

collections=[]


In [15]:
meta

{'자치법규ID': '2233372',
 '자치법규명': '횡성군 수도급수 조례',
 '자치법규일련번호': '2058483',
 '지자체기관명': '강원특별자치도 횡성군',
 '시행일자': '20250714',
 '제개정정보': '일부개정',
 '자치법규종류': 'C0001',
 '공포번호': '2855',
 '전화번호': '033-340-5821',
 '담당부서명': '상하수도사업소',
 '자치법규발의종류': '',
 '공포일자': '20250714'}

In [16]:
len(" ".join(ctx_list))

13352

In [17]:
from langchain_core.documents import Document

documents = []
chunk = ""
for item in ctx_list:
    if len(chunk) < 500:
        chunk += "\n" + item
    else:
        chunk = chunk.strip()
        chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + chunk
        doc = Document(
            page_content=chunk,
            metadata=meta
        )
        documents.append(doc)
        chunk = item
chunk = chunk.strip()
chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + chunk
doc = Document(
    page_content=chunk,
    metadata=meta
)
documents.append(doc)
documents

[Document(metadata={'자치법규ID': '2233372', '자치법규명': '횡성군 수도급수 조례', '자치법규일련번호': '2058483', '지자체기관명': '강원특별자치도 횡성군', '시행일자': '20250714', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2855', '전화번호': '033-340-5821', '담당부서명': '상하수도사업소', '자치법규발의종류': '', '공포일자': '20250714'}, page_content='지자체기관명: 강원특별자치도 횡성군, 자치법규명: 횡성군 수도급수 조례\n제1장 총칙제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.\n제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음 각 호와 같다.1. "급수설비"란 수도사업자가 일반의 수요자에게 원수 또는 정수를 공급하기 위하여 설치한 배수관으로부터 분기하여 설치된 급수관(옥내급수관을 포함한다)·계량기·저수조·수도꼭지·그 밖의 급수를 위하여 필요한 기구(器具)를 말한다.2. "급수공사"란 급수설비의 신설, 개조, 수선 및 철거 등의 공사를 말한다.3. "수도사용자 등"이란 급수설비의 사용자, 소유자 및 관리인 등을 말한다.4. “중수도”란 사용한 수돗물을 생활용수·공업용수 등으로 재활용할 수 있도록 다시 처리하는 시설을 말한다.5. "공업용수"란 정수장에서 1차 침전 처리한 용수를 말한다.6. “호(戶)”란 독립적으로 생활을 영위할 수 있도록 건립된 건축단위를 말한다.7. “교육시설”이란「초·중등교육법」 제2조 각 호에 따른 학교 및 「유아교육법」 제2조제2호에 따른 유치원에서 사용하는 시설을 말한다.<개정 2019. 7. 15.>'),
 Document(metadata={'자치법규ID':

In [33]:
len(df["자치법규ID"])

1603

In [14]:
from tqdm import tqdm
from langchain_core.documents import Document

documents = []
all_ctx_list = []

for x in tqdm(range(len(df["자치법규ID"]))):
    id = df["자치법규ID"][x]
    url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()

    meta = data["LawService"]["자치법규기본정보"]
    content = data["LawService"]["조문"]["조"]

    ctx_list = []
    header = ""
    for ctx in content:
        if ctx["조문여부"] == "Y":
            ctx_list.append(header + ctx["조내용"])
            all_ctx_list.append(header + ctx["조내용"])
            header = ""
        else:
            header = ctx["조내용"] + " "

    chunk = ""
    cnt = 1
    for item in ctx_list:
        if len(chunk) < 500:
            chunk += "\n" + item
        else:
            chunk = chunk.strip()
            chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + chunk
            doc = Document(
                page_content=chunk,
                metadata=meta,
                id = f"{meta['자치법규ID']}_chunk_{cnt}"
            )
            cnt += 1
            documents.append(doc)
            chunk = item
    # 마지막 청크 저장 전에 확인
    if chunk.strip():  # 빈 청크가 아닐 때만 저장
        chunk = chunk.strip()
        chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + chunk
        doc = Document(
            page_content=chunk,
            metadata=meta,
            id = f"{meta['자치법규ID']}_chunk_{cnt}"
        )
        documents.append(doc)   

100%|██████████| 1603/1603 [04:31<00:00,  5.90it/s]


In [117]:
from langchain_community.embeddings import ClovaXEmbeddings

clova_embedder = ClovaXEmbeddings(
                model="bge-m3",
                api_key=os.getenv("CLOVASTUDIO_API_KEY")
                )

In [120]:
clova_embedder.

ClovaXEmbeddings(client=<httpx.Client object at 0x128bd4370>, async_client=<httpx.AsyncClient object at 0x128bd4ca0>, ncp_clovastudio_api_key=SecretStr('**********'), ncp_apigw_api_key=None, base_url='https://clovastudio.stream.ntruss.com', app_id=None, service_app=False, model_name='bge-m3', timeout=60)

In [ ]:
collection_name = "local_regulations"

# 컬렉션 생성 (벡터 차원은 임베딩 모델에 따라 조정)
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=clova_embedder, distance=Distance.COSINE),  # OpenAI ada-002 기준
)


In [33]:
data["LawService"]["부칙"]

{'부칙공포일자': '20250714',
 '부칙내용': '부칙  ①(시행일) 이 조례는 공포한 날로부터 시행한다.②(폐지조례) 이 조례 시행과 동시 횡성군수도급수조례(1979.6.23 조례제647호)는 이를 폐지한다.③(경과조치) 이 조례 시행전에 종전 조례에 의하여 행하여진 승인 및 처분사항은 이 조례에 의한 것으로 본다.④(가산금 등에 대한 경과조치) 이 조례 시행전에 검침 및 체납된 수도사용요금 및 급수장치 손료가산금에 대하여는 종전의 규정에 의한다부칙 <2001.5.8.> 이 조례는 공포한 날로부터 시행한다.부칙 <2003.1.27.> 제1조(시행일) (생략)제2조(한시기구에 대한 경과규정) (생략)제3조(다른 조례의 개정) 제1항 내지 제11항 (생략)⑫횡성군수도급수조례중 다음과 같이 개정한다.제9조제2항중 “상하수도 담당주사”를 “상수도 담당주사”로 한다.제13항 내지 제18항 (생략)부칙 <2003.6.2.> 이 조례는 공포한 날로부터 시행한다.부칙 <2005.1.11. 조례 제1846호> ①(시행일) 이 조례는 공포한 날부터 시행한다.②(상수도요금의 적용시기) 제33조1항과 관련하여 [별표2]의 상수도요금, 제34조제1항과 관련하여 [별표 3]의 업종구분, 제39조제2항과 관련하여 [별표4]의 급수장치 손료에 대하여는 2004년 월분 상수도 요금부과 분부터 적용한다.③(일반적 경과조치) 이 조례 시행당시 종전의 규정에 의하여 접수 처리 중인 급수공사에 대하여는 종전의 규정에 의한다.부칙 <2006.1.12. 조례 제1881호> ①(시행일) 이 조례는 공포한 날부터 시행한다.②(상수도요금의 적용시기) 제33조제1항과 관련하여 별표 2의 업종별 요금표, 제39조제2항과 관련하여 별표 4의 급수장치 손료에 대하여 2006년 1월분 수도요금 부과분부터 적용한다.③(일반적 경과조치) 이 조례 시행당시 종전의 규정에 의하여 접수 처리중인 급수공사에 대하여는 종전의 규정에 의한다.부칙 <2007.1.15. 조례 제1919호> ①(시행일) 이 조례는 공포한 

In [32]:
data["LawService"]["별표"]

{'별표단위': {'별표제목': '별표',
  '별표첨부파일구분': 'hwp',
  '별표첨부파일명': 'http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=154521627&flNm=%EB%B3%84%ED%91%9C',
  '별표번호': '0001',
  '별표키': '20574529',
  '별표내용': '',
  '별표구분': '서식',
  '별표가지번호': '00'}}

In [ ]:
from langchain_teddynote.document_loaders import HWPLoader

# HWP Loader 객체 생성
loader = HWPLoader("http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=154521627&flNm=%EB%B3%84%ED%91%9C")
# 문서 로드
docs = loader.load()

In [106]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

model = ChatOllama(
    model="gpt-oss:20b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)
system_message = """당신은 법률 문서의 내용만 보고, 참고해야하는 법률 문서와 각 조항 리스트를 추출해야합니다.
참고해야하는 법률 문서가 있다면, 해당 제목은 반드시 내용 안에 포함되어 있습니다.
참고해야하는 세부 조항만 언급되어 있다면, "기타" key에 조항 번호, 별표 리스트를 넣어주세요.
추측하지 마세요.

출력 형식은 반드시 아래의 JSON 형식을 따라야합니다.
관련 세부 조항이 없다면 빈 리스트로 반환하세요.

"법률 문서 제목": ["관련 조항 번호", "..."],
"기타": ["별표", "조항 번호", "..."]
"""

system_message = """당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.

아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.
2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.
3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "기타" key에 넣습니다.
4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다 (파싱 가능한 구조).

출력 형식:
  "법률 문서 제목": ["관련 조항 번호", "..."],
  "기타": ["별표", "조항 번호", "..."]

주의:
- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.
- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.
- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{question}")
])

chain = prompt | model

# 일반 Chain 생성
chain = prompt | model | JsonOutputParser()
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.\n\n아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.\n2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.\n3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "기타" key에 넣습니다.\n4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다 (파싱 가능한 구조).\n\n다음은 약어입니다. 실제 문서에서 언급된 약어를 이와 매칭하여 사용하세요:\n- \'법\': 건축물관리법\n- \'영\' : 건축물관리법 시행령\n\n출력 형식:\n  "법률 문서 제목": ["관련 조항 번호", "..."],\n  "기타": ["별표", "조항 번호", "..."]\n\n주의:\n- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.\n- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.\n- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:20b-cloud', te

In [137]:
model_abb = ChatOllama(
    model="gpt-oss:20b-cloud",
    temperature=0.1,
    # max_tokens = 1024*4,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message_abb = """당신은 법률 문서 전문을 분석하여, 그 안에서 약어와 축약어의 실제 명칭을 파악 후 추출해야합니다.

아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 명시적으로 등장한 약어 또는 축약어만 추출합니다.
2. 약어가 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.
4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다 (파싱 가능한 구조).

출력 형식:
  "약어": "실제 명칭"

주의:
- 약어가 문서에 없다면 빈 JSON 객체로 반환합니다.
- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.
"""

prompt_abb = ChatPromptTemplate.from_messages([
    ("system", system_message_abb),
    ("user", "{question}")
])

# 일반 Chain 생성
chain_abb = prompt_abb | model_abb | JsonOutputParser()
chain_abb

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 약어와 축약어의 실제 명칭을 파악 후 추출해야합니다.\n\n아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 명시적으로 등장한 약어 또는 축약어만 추출합니다.\n2. 약어가 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.\n4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다 (파싱 가능한 구조).\n\n출력 형식:\n  "약어": "실제 명칭"\n\n주의:\n- 약어가 문서에 없다면 빈 JSON 객체로 반환합니다.\n- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:20b-cloud', temperature=0.1)
| JsonOutputParser()

In [142]:
all_chunks = " ".join(ctx_list[:5])
print(all_chunks)
len(all_chunks), len(ctx_list)

[제1장 총칙] 제1조(목적) 이 조례는「국토의 계획 및 이용에 관한 법률」, 같은 법 시행령, 같은 법 시행규칙 및 관계 법령에서 위임된 사항과 그 시행에 필요한 사항을 규정하는 것을 목적으로 한다. [제1장 총칙] 제2조(국토이용 및 관리의 기본방향) 강릉시(이하 "시"라 한다)의 국토이용 및 관리는「국토의 계획 및 이용에 관한 법률」(이하 “법”이라 한다) 제3조의 기본원칙을 바탕으로 자연환경의 보전과 자원의 효율적 활용을 통하여 환경적으로 건전하고 지속가능한 발전을 이루면서 접근성ㆍ편의성ㆍ안정성을 고려한 도시계획을 수립ㆍ집행하는 것을 기본방향으로 한다. <개정 2024.9.25.> [제2장 도시기본계획] 제3조(도시기본계획의 위상) 법 제22조의2에 따라 강원특별자치도지사의 승인을 받은 도시기본계획은 관할구역 안에서 강릉시장(이하 “시장”이라 한다)이 수립하는 도시개발 및 도시관리 등에 관한 각종 계획의 기본이 된다.<개정 2012. 12. 26., 2023.6.7., 2024.9.25.> [제2장 도시기본계획] 제4조(추진기구 및 공청회 등)① 시장은 법 제18조에 따라 도시기본계획을 수립하고자 할 경우에는 별도의 추진기구 및 전문가로 구성된 자문단을 한시적으로 운영할 수 있다. <개정 2012. 12. 26., 2024.9.25.>② 도시기본계획의 수립을 위한 공청회는 자문단의 자문을 거친 이후에 개최 할 수 있다.(자문단을 설치한 경우로 한정한다) <신설 2012. 12. 26.> <개정 2024.9.25.>③ 제2항에 따른 자문으로 도시기본계획 승인 요청을 위한 강릉시 도시계획위원회(이하 “도시계획위원회”라 한다)의 자문을 갈음할 수 없다. <신설 2012. 12. 26.>[본조제목개정 2024.9.25.] [제2장 도시기본계획] 제5조(도시기본계획의 수립을 위한 공청회 개최방법 등)① 시장은 「국토의 계획 및 이용에 관한 법률 시행령」(이하“영”이라 한다) 제12조제4항에 따라 도시기본계획에 대한 공청회(이하 “공청회”라 한다)를 개최하고자 할

(1491, 102)

In [132]:
chain_abb.invoke({"question": all_chunks})

{'시': '강릉시', '법': '국토의 계획 및 이용에 관한 법률', '영': '국토의 계획 및 이용에 관한 법률 시행령'}

In [134]:
chain_abb.invoke({"question": all_chunks})

{'인허가등': '인허가·허가·승인·협의 또는 신고 등'}

In [141]:
chain_abb.invoke({"question": all_chunks})

{'시': '강릉시',
 '법': '국토의 계획 및 이용에 관한 법률',
 '시장': '강릉시장',
 '도시계획위원회': '강릉시 도시계획위원회',
 '영': '국토의 계획 및 이용에 관한 법률 시행령'}

In [73]:
from langchain_core.documents import Document
import tqdm

documents = []
for item in tqdm.tqdm(ctx_list):
    chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + item
    q = "법률 문서의 내용은 다음과 같습니다: " + item
    links_llm = chain.invoke({"question": q})
    links_llm = {k: v for k, v in links_llm.items() if v}
    if links_llm:
        meta = meta | {"links" : [links_llm]}
    doc = Document(
        page_content=chunk,
        metadata=meta,
        links = links_llm
    )
    documents.append(doc)
documents

 50%|█████     | 1/2 [00:03<00:03,  3.38s/it]

{'수도법': ['제38조'], '지방자치법': ['제136조', '제139조제1항'], '지방공기업법': ['제22조']}


100%|██████████| 2/2 [00:06<00:00,  3.02s/it]

{'초·중등교육법': ['제2조'], '유아교육법': ['제2조제2호']}


[Document(metadata={'자치법규ID': '2233372', '자치법규명': '횡성군 수도급수 조례', '자치법규일련번호': '2058483', '지자체기관명': '강원특별자치도 횡성군', '시행일자': '20250714', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2855', '전화번호': '033-340-5821', '담당부서명': '상하수도사업소', '자치법규발의종류': '', '공포일자': '20250714', 'links': [{'수도법': ['제38조'], '지방자치법': ['제136조', '제139조제1항'], '지방공기업법': ['제22조']}]}, page_content='지자체기관명: 강원특별자치도 횡성군, 자치법규명: 횡성군 수도급수 조례\n[제1장 총칙] 제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.'),
 Document(metadata={'자치법규ID': '2233372', '자치법규명': '횡성군 수도급수 조례', '자치법규일련번호': '2058483', '지자체기관명': '강원특별자치도 횡성군', '시행일자': '20250714', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2855', '전화번호': '033-340-5821', '담당부서명': '상하수도사업소', '자치법규발의종류': '', '공포일자': '20250714', 'links': [{'초·중등교육법': ['제2조'], '유아교육법': ['제2조제2호']}]}, page_content='지자체기관명: 강원특별자치도 횡성군, 자치법규명: 횡성군 수도급수 조례\n[제1장 총칙] 제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음 각 호와 같다.1. "급수설비"

In [125]:
x

11

In [ ]:
from tqdm import tqdm
from langchain_core.documents import Document

# documents = []
# all_ctx_list = []

for x in tqdm(range(11, len(df["자치법규ID"]))):
# for x in tqdm(range(1)):
    id = df["자치법규ID"][x]
    url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()

    meta = data["LawService"]["자치법규기본정보"]
    content = data["LawService"]["조문"]["조"]

    ctx_list = []
    header = ""
    for ctx in content:
        if ctx["조문여부"] == "Y":
            txt = header + " " + ctx["조내용"]
            txt = txt.strip()
            ctx_list.append(txt)
            all_ctx_list.append(txt)
        else:
            header = "[" + ctx["조내용"] + "]"

    cnt = 1
    for item in ctx_list:
        chunk = "지자체기관명: " + meta["지자체기관명"] + ", 자치법규명: " + meta["자치법규명"] + "\n" + item
        q = "법률 문서의 내용은 다음과 같습니다: " + item
        links_llm = chain.invoke({"question": q})
        print(q)
        print(links_llm)
        filtered = {
            k: v for k, v in links_llm.items()
            if (k in ["기타", "법률 문서 제목"] and v) or (k not in ["기타", "법률 문서 제목"])
        }
        meta_with_links = meta.copy()
        if filtered:
            meta_with_links["links"] = [filtered]
        doc = Document(
            page_content=chunk,
            metadata=meta_with_links,
            id = f"{meta['자치법규ID']}_chunk_{cnt}"
        )
        cnt += 1
        documents.append(doc)

In [123]:
import json

# with open("documents.jsonl", "w", encoding="utf-8") as f:
#     for doc in documents:
#         json.dump({
#             "id": doc.id,
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         }, f, ensure_ascii=False)
#         f.write("\n")


In [116]:
all_ctx_list

['제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.',
 '제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다), 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 모든 차의 교통에 사용하도록 된 도로의 부분을 말한다.3. “보도(步道)”라 함은 연석선, 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 보행자(유모차 및 신체장애인용 의자차를 포함한다)의 통행에 사용하도록 되어있는 도로의 부분을 말한다. <개정 2017.3.6.>4. “이면도로”라 함은 「도로법」에 따른 고속국도ㆍ일반국도ㆍ지방도ㆍ군도 및 농어촌도로(「농어촌도로정비법」에 따른 도로를 말한다)가 아닌 일반의 교통에 사용되는 도로로서 차도와 보도의 구분이 없는 폭 12미터 미만의 도로를 말한다.5. “보행자전용도로”라 함은 보행자만이 다닐 수 있도록 안전표지 및 그 밖에 이와 비슷한 공작물로써 표시한 도로를 말한다.6. “제설ㆍ제빙작업”이라 함은 도로상의 눈 또는 얼음을 제거하거나 녹게 하는 재료 및 모래 등을 뿌려서 보행자와 차량의 안전한 통행에 지장이 없도록 하는 것을 말한다.7. “건축물관리자”라 함은 건축물의 소유자ㆍ점유자 또는 관리자로서 그 건축물의 관리 책임이 있는 사람을 말한다.8. “시설물의 지붕”이라 함은 「자연재해대책법 시행령」제22조의8에 따른 시설물의 지붕을 말한다. <신설 2017.7.31.>',
 '제3조(건축물관리자의 제설ㆍ제빙 책임) 건축물관리자는 관리하고 있는 건축물의 대지에 접한 보도, 이면도로, 보행자전용도로, 시설물의 지붕에 대한 제설ㆍ제빙작업을 하여야 한다. <신설

In [37]:
q = "법률 문서의 내용은 다음과 같습니다: " + ctx_list[0]
print(q)

법률 문서의 내용은 다음과 같습니다: 제1장 총칙 제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.


In [38]:
chain.invoke({"question": q})

{'수도법': ['제38조'], '지방자치법': ['제136조', '제139조제1항'], '지방공기업법': ['제22조'], '기타': []}

## 목록 조회

- target: 서비스 대상
    - `eflaw`: 현행법령(시행일)

In [ ]:
# api로 현행법령(시행일) 목록 조회하면 누락되는게 많음.
url = f"http://www.law.go.kr/DRF/lawSearch.do?OC={oc}&target=law&type=JSON&query=건축"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

In [ ]:
print(len(data["LawSearch"]))
data["LawSearch"]

In [86]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID=010787&type=JSON"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙대통령령 제35811호(2025.10.1)', '정부조직 개편 반영을 위한 306개 법령의 일부개정에 관한 대통령령', '제1조부터 제169조까지 생략', '제170조(「건축기본법 시행령」의 개정) 건축기본법 시행령 일부를 다음과 같이 개정한다.', '  제5조제10호 및 제12호를 각각 다음과 같이 한다.', '    10. 산업통상부장관', '    12. 기후에너지환경부장관', '  제18조제1항 중 "산업통상자원부장관"을 "산업통상부장관"으로 한다.', '  제23조제3항제1호사목을 다음과 같이 한다.', '      사. 산업통상부', '제171조부터 제306조까지 생략', '          부칙', '이 영은 공포한 날부터 시행한다. 다만, 이 영에 따라 개정되는 대통령령 중 이 영 시행 전에 공포되었으나 시행일이 도래하지 아니한 대통령령을 개정한 부분은 각각 해당 대통령령의 시행일부터 시행한다.']]}, '법령키': '0107872025100135811', '기본정보': {'법령명_한글': '건축기본법 시행령', '공포번호': '35811', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010787', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '대통령령', '법종구분코드': 'A0007'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '391623', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법 시행령', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': 

In [ ]:
url = f"https://www.law.go.kr/DRF/lawSearch.do?OC={oc}&target=lsStmd&type=JSON&query=건축"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

### HTML

In [ ]:
from bs4 import BeautifulSoup

url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=HTML"

# 1️⃣ URL 요청
response = requests.get(url)
response.raise_for_status()
html = response.text  # HTML 본문

# 2️⃣ BeautifulSoup으로 파싱
soup = BeautifulSoup(html, "html.parser")

In [38]:
# 3️⃣ iframe 추출
iframe = soup.find("iframe")
if iframe:
    iframe_url = iframe.get("src")
    print("실제 법령 본문 URL:", iframe_url)
else:
    print("iframe을 찾을 수 없습니다.")

실제 법령 본문 URL: http://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923&chrClsCd=010202&urlMode=lsInfoP&efYd=&ancYnChk=0&mobile=&gubun=api


In [39]:
iframe_url

'http://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923&chrClsCd=010202&urlMode=lsInfoP&efYd=&ancYnChk=0&mobile=&gubun=api'

In [40]:
inner_html = requests.get(iframe_url).text
inner_soup = BeautifulSoup(inner_html, "html.parser")

# 실제 법령 본문 조문 추출 (예시)
for article in inner_soup.select(".pgroup"):
    print(article.get_text(strip=True))

In [58]:
res = requests.post("https://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923#0000")
soup = BeautifulSoup(res.text, "html.parser")
law_texts = [x.get_text(strip=True) for x in soup.select(".pgroup, .pgroup2, .pgroup3")]
print(law_texts[:5])

[]


In [53]:
'<div id="lsBdy">' in res.text

False

In [23]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙법률 제21065호(2025.10.1)', '정부조직법 일부개정법률', '[본문 생략]', '        부칙', '제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터ㆍㆍㆍ<생략>ㆍㆍㆍ 시행한다.', '  1. 및 2. 생략', '제2조부터 제6조까지 생략', '제7조(다른 법률의 개정) ①부터 <436>까지 생략', '  <437> 건축기본법 일부를 다음과 같이 개정한다.', '  제20조 각 호 외의 부분 및 제21조제1항ㆍ제2항 중 "산업통상자원부장관"을 각각 "산업통상부장관"으로 한다.', '  <438>부터 <626>까지 생략', '제8조 생략']]}, '법령키': '0105942025100121065', '기본정보': {'법령명_한글': '건축기본법', '공포번호': '21065', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010594', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '법률', '법종구분코드': 'A0002'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '388805', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': [{'부칙키': '2007122108783', '부칙공포일자': '20071221', '부칙내용': [['부칙 <제8783호,20

In [26]:
data.keys()

dict_keys(['법령'])

In [27]:
data['법령'].keys()

dict_keys(['개정문', '법령키', '기본정보', '부칙', '조문', '제개정이유'])

In [30]:
data

False

# 법령

In [5]:
import requests
import os
import pandas as pd

In [6]:
oc = os.getenv('OC', '')

In [20]:
df = pd.read_csv("data/법령검색목록_법령_건축.csv", skiprows=1)
df.head()

,순번,법령MST,소관부처코드,소관부처명,법령ID,법령명,공포일자,공포번호,시행일자,법령구분코드,법령구분명,법령분야코드,법령분야명
0,1,276923,1613000,국토교통부,10594,건축기본법,20251001,제21065호,20251001,A0002,법률,34030000,第3章 建 築
1,2,277915,1613000,국토교통부,10787,건축기본법 시행령,20251001,제35811호,20251001,A0007,대통령령,34030000,第3章 建 築
2,3,266691,1613000,국토교통부,13478,건축물관리법,20241203,제20549호,20250604,A0002,법률,34030000,第3章 建 築
3,4,271337,1613000,국토교통부,13742,건축물관리법 시행령,20250527,제35549호,20250604,A0007,대통령령,34030000,第3章 建 築
4,5,271531,1613000,국토교통부,13747,건축물관리법 시행규칙,20250602,제1495호,20250602,A0103,국토교통부령,34030000,第3章 建 築


In [44]:
import json
from langchain_core.documents import Document

def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
                
            data = json.loads(line)
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    return documents

In [45]:
documents = load_documents_from_jsonl("documents.jsonl")
documents

[Document(id='2019610_clause_1', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', '약어': {}, 'links': [{'자연재해대책법': ['제27조제2항']}]}, page_content='제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.'),
 Document(id='2019610_clause_2', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', '약어': {}, 'links': [{'도로법': [], '농어촌도로정비법': [], '자연재해대책법 시행령': ['제22조의8']}]}, page_content='제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연

In [9]:
df.iloc[0,:]

순번               1
법령MST       276923
소관부처코드     1613000
소관부처명        국토교통부
법령ID         10594
법령명          건축기본법
공포일자      20251001
공포번호       제21065호
시행일자      20251001
법령구분코드       A0002
법령구분명           법률
법령분야코드    34030000
법령분야명      第3章 建 築
Name: 0, dtype: object

In [21]:
for n, name in enumerate(df["법령명"]):
    if name == "수도법":
        print(df.iloc[n,:])
        break

In [26]:
id = df["법령ID"][0]
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙법률 제21065호(2025.10.1)', '정부조직법 일부개정법률', '[본문 생략]', '        부칙', '제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터ㆍㆍㆍ<생략>ㆍㆍㆍ 시행한다.', '  1. 및 2. 생략', '제2조부터 제6조까지 생략', '제7조(다른 법률의 개정) ①부터 <436>까지 생략', '  <437> 건축기본법 일부를 다음과 같이 개정한다.', '  제20조 각 호 외의 부분 및 제21조제1항ㆍ제2항 중 "산업통상자원부장관"을 각각 "산업통상부장관"으로 한다.', '  <438>부터 <626>까지 생략', '제8조 생략']]}, '법령키': '0105942025100121065', '기본정보': {'법령명_한글': '건축기본법', '공포번호': '21065', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010594', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '법률', '법종구분코드': 'A0002'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '388805', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': [{'부칙키': '2007122108783', '부칙공포일자': '20071221', '부칙내용': [['부칙 <제8783호,20

In [27]:
data["법령"].keys()

dict_keys(['개정문', '법령키', '기본정보', '부칙', '조문', '제개정이유'])

In [28]:
data["법령"]["기본정보"]

{'법령명_한글': '건축기본법',
 '공포번호': '21065',
 '전화번호': '044-201-3766',
 '언어': '한글',
 '제개정구분': '타법개정',
 '법령ID': '010594',
 '공동부령정보': '',
 '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'},
 '공포법령여부': 'N',
 '법종구분': {'content': '법률', '법종구분코드': 'A0002'},
 '시행일자': '20251001',
 '연락부서': {'부서단위': {'부서연락처': '044-201-3766',
   '부서키': '388805',
   '부서명': '건축정책과',
   '소관부처명': '국토교통부',
   '소관부처코드': '1613000'}},
 '법령명_한자': '건축기본법',
 '법령명약칭': '',
 '공포일자': '20251001',
 '편장절관': '34030000'}

In [35]:
content = data["법령"]["조문"]["조문단위"]
content[:10]

[{'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001000',
  '조문내용': '            제1장 총칙',
  '조문제목': '',
  '조문이동이후': '',
  '조문여부': '전문'},
 {'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001001',
  '조문내용': '제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
  '조문제목': '목적',
  '조문이동이후': '',
  '조문여부': '조문'},
 {'조문번호': '2',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0002001',
  '항': {'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
    {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
    {'호번호': '3.',
     '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]},
  '조문내용': '제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.',
  '조문제목': '기본이념',
  '조문이동이후': '',
  '조문여부': '조문'},
 {'조문번호': '3',
  '조문제개정

In [38]:
content[1]

{'조문번호': '1',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0001001',
 '조문내용': '제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
 '조문제목': '목적',
 '조문이동이후': '',
 '조문여부': '조문'}

In [37]:
content[2]

{'조문번호': '2',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0002001',
 '항': {'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
   {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
   {'호번호': '3.',
    '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]},
 '조문내용': '제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.',
 '조문제목': '기본이념',
 '조문이동이후': '',
 '조문여부': '조문'}

In [34]:
ctx_list = []
ctx_org = []
header = ""
for ctx in content:
    ctx_txt = ctx["조문내용"].strip()
    if ctx["조문여부"] == "조문": # 조
        txt = header + " " + ctx_txt
        ctx_list.append(txt.strip())
        # header = ""
    else: # 장, 항, 
        header = "[" + ctx_txt + "]"
    ctx_org.append(ctx_txt)
ctx_list

['[제1장 총칙] 제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
 '[제1장 총칙] 제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.',
 '[제1장 총칙] 제3조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.',
 '[제1장 총칙] 제4조(국가 및 지방자치단체의 책무)',
 '[제1장 총칙] 제5조(국민의 의무)',
 '[제1장 총칙] 제6조(다른 법률과의 관계) 국가는 건축에 관한 다른 법률을 제정 또는 개정하는 경우에는 이 법의 목적과 기본이념에 맞도록 하여야 한다.',
 '[제2장 건축정책의 기본방향] 제7조(건축의 생활공간적 공공성 구현)',
 '[제2장 건축정책의 기본방향] 제8조(건축의 사회적 공공성 확보)',
 '[제2장 건축정책의 기본방향] 제9조(건축의 문화적 공공성 실현)',
 '[제3장 건축정책의 수립] 제10조(건축정책기본계획의 수립)',
 '[제3장 건축정책의 수립] 제11조(건축정책기본계획의 내용) 건축정책기본계획에는 다음 각 호의 사항이 포함되어야 한다.',
 '[제3장 건축정책의 수립] 제12조(지역건축기본계획의 수립 등)',
 '[제4장 건축정책위원회] 제13조(국가건축정책위원회)',
 '[제4장 건축정책위원회] 제14조(국가건축정책위원회의 기능) 국가건축정책위원회는 다음 각 호의 사항을 심의한다. <개정 2020.6.9>',
 '[제4장 건축정책위원회] 제15조(건축정책 국회보고)',
 '[제4장 건축정책위원회] 제16조(건축 기본조사)',
 '[제4장 건축정책위원회] 제17조(기획단)',
 '[제4장 건축정책위원회] 제18조(지역건축위원회)',
 '[제4장 건축정책위원회] 제19조(지역건축위원회의 기능) 지역건축위원회는 다음 각 호의 사항에 관한 사무를 행한다.',